In [ ]:
# Check for infinite or extreme values
inf_count = np.isinf(all_features.select_dtypes(include=[np.number])).sum()
if inf_count.sum() > 0:
    print("⚠️ Features with infinite values:")
    print(inf_count[inf_count > 0])
else:
    print("✅ No infinite values detected")

# Check for extreme outliers (beyond 5 std deviations)
print("\n📊 Extreme Value Detection (>5σ):")
for col in all_features.select_dtypes(include=[np.number]).columns:
    mean = all_features[col].mean()
    std = all_features[col].std()
    outliers = ((all_features[col] - mean).abs() > 5 * std).sum()
    if outliers > 0:
        print(f"  {col}: {outliers} extreme values ({outliers/len(all_features)*100:.2f}%)")

In [ ]:
# Missing value analysis
missing_pct = (all_features.isnull().sum() / len(all_features) * 100).sort_values(ascending=False)
missing_features = missing_pct[missing_pct > 0]

if len(missing_features) > 0:
    print("📉 Features with Missing Values:")
    print(missing_features)

    # Visualize missing data pattern
    plt.figure(figsize=(12, 6))
    sns.heatmap(
        all_features[missing_features.index].isnull(), cbar=True, cmap="viridis", yticklabels=False
    )
    plt.title("Missing Data Pattern", fontsize=14)
    plt.xlabel("Features")
    plt.tight_layout()
    plt.show()
else:
    print("✅ No missing values in dataset")

## 9. Save Processed Features

Export the engineered features to multiple formats for downstream analysis and model training.

In [ ]:
# Create output directory
output_dir = project_root / 'data' / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

# Save to multiple formats
# 1. Parquet (efficient, preserves dtypes)
parquet_path = output_dir / f'{SYMBOL}_engineered_features.parquet'
all_features.to_parquet(parquet_path, compression='snappy')
print(f"✅ Saved to Parquet: {parquet_path}")

# 2. CSV (human-readable)
csv_path = output_dir / f'{SYMBOL}_engineered_features.csv'
all_features.to_csv(csv_path)
print(f"✅ Saved to CSV: {csv_path}")

# 3. HDF5 (for large datasets)
h5_path = output_dir / f'{SYMBOL}_engineered_features.h5'
all_features.to_hdf(h5_path, key='features', mode='w', complevel=9)
print(f"✅ Saved to HDF5: {h5_path}")

# Save feature metadata
metadata = {
    'symbol': SYMBOL,
    'data_source': 'OpenBB Terminal',
    'date_range': {
        'start': START_DATE,
        'end': END_DATE,
    },
    'n_samples': len(all_features),
    'n_features': len(all_features.columns),
    'time_range': {
        'start': str(all_features.index.min()),
        'end': str(all_features.index.max()),
    },
    'features': {
        'microstructure': [col for col in all_features.columns if any(
            x in col for x in ['imbalance', 'spread', 'vpin', 'microprice', 'realized_vol']
        )],
        'options': [col for col in all_features.columns if any(
            x in col for x in ['iv_skew', 'term', 'vol_of_vol', 'delta', 'gamma', 'vega']
        )],
        'market': [col for col in all_features.columns if col in ['price', 'returns', 'log_returns']],
    },
    'missing_values': missing_pct.to_dict() if 'missing_pct' in locals() else {},
    'correlations_high': high_corr_df.to_dict('records') if 'high_corr_df' in locals() and len(high_corr_pairs) > 0 else []
}

import json

metadata_path = output_dir / f'{SYMBOL}_features_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Saved metadata: {metadata_path}")

print(f"\n📦 Feature Engineering Complete!")
print(f"   Symbol: {SYMBOL}")
print(f"   Total Features: {len(all_features.columns)}")
print(f"   Total Samples: {len(all_features)}")
print(f"   Output Directory: {output_dir}")print(f"   Output Directory: {output_dir}")

## Summary & Next Steps

**Microstructure Features Engineered:**
- Order book imbalance (levels 1, 5, 10)
- Microprice (volume-weighted mid)
- Spread dynamics (absolute, relative, normalized)
- VPIN (trade flow toxicity)
- Realized volatility (5, 20, 60 period windows)

**Options Features Engineered:**
- IV skew (30d, 60d maturities)
- Term structure slope and curvature
- Vol-of-vol (volatility of implied volatility)
- Volatility smile curvature
- Greeks exposure (delta, gamma, vega)

**Data Source:**
- ✅ Real market data from OpenBB Terminal
- ✅ Yahoo Finance (free tier)
- ✅ Optional: Polygon, Alpha Vantage, CBOE with API keys

**Data Quality:**
- ✅ No constant features
- ✅ No infinite values
- ✅ Correlations analyzed
- ✅ Temporal stability checked
- ✅ Ready for model training

**Next Steps:**
1. **Notebook 02**: Generate volatility labels and trading targets
2. **Notebook 03**: Build microstructure trading signals
3. **Notebook 04**: Test RL environment with engineered features
4. **Notebook 05**: Train and evaluate RL agents (PPO, DDPG, etc.)

In [ ]:
# Import libraries
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Project modules
from src.data.loader import DataLoader, MarketDataLoader
from src.data.preprocess import DataPreprocessor
from src.data.feature_engineering import (
    MicrostructureFeatureEngineer,
    OptionsFeatureEngineer,
    FeatureEngineeringPipeline
)

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("✅ All imports successful!")
print(f"Project root: {project_root}")

### 1.1 Configure Data Loading

Set symbol, date range, and data provider preferences.

In [ ]:
# Generate synthetic order book data
np.random.seed(42)

# Time range
n_samples = 10000
dates = pd.date_range("2024-01-01", periods=n_samples, freq="1min")

# Base price with realistic dynamics
base_price = 100
returns = np.random.normal(0, 0.0002, n_samples)
prices = base_price * np.exp(np.cumsum(returns))

# Order book levels (10 levels)
order_book_data = []

for i, (date, price) in enumerate(zip(dates, prices)):
    # Spread that varies with volatility
    volatility = np.std(returns[max(0, i - 20) : i + 1])
    spread = max(0.01, price * volatility * 2)

    # Generate 10 levels
    for level in range(1, 11):
        bid_price = price - spread * level / 2
        ask_price = price + spread * level / 2

        # Volume decreases with level
        base_volume = np.random.exponential(1000)
        bid_volume = base_volume * (1.1 - level * 0.05) * np.random.uniform(0.8, 1.2)
        ask_volume = base_volume * (0.9 + level * 0.05) * np.random.uniform(0.8, 1.2)

        order_book_data.append(
            {
                "timestamp": date,
                "level": level,
                f"bid_price_{level}": bid_price,
                f"ask_price_{level}": ask_price,
                f"bid_size_{level}": bid_volume,
                f"ask_size_{level}": ask_volume,
            }
        )

# Create DataFrame and pivot
ob_df = pd.DataFrame(order_book_data)
ob_pivot = ob_df.groupby("timestamp").first().reset_index()

# Add aggregate columns
ob_pivot["mid_price"] = (ob_pivot["bid_price_1"] + ob_pivot["ask_price_1"]) / 2
ob_pivot["spread"] = ob_pivot["ask_price_1"] - ob_pivot["bid_price_1"]
ob_pivot["total_bid_size"] = ob_pivot[[f"bid_size_{i}" for i in range(1, 11)]].sum(axis=1)
ob_pivot["total_ask_size"] = ob_pivot[[f"ask_size_{i}" for i in range(1, 11)]].sum(axis=1)

print(f"Generated order book data: {len(ob_pivot)} snapshots")
print(f"Shape: {ob_pivot.shape}")
ob_pivot.head()

In [ ]:
# Generate synthetic trade data
trade_data = []

for i, (date, price) in enumerate(zip(dates, prices)):
    # Random number of trades per minute
    n_trades = np.random.poisson(5)

    for _ in range(n_trades):
        # Trade price around mid
        trade_price = price * (1 + np.random.normal(0, 0.0001))
        trade_size = np.random.exponential(100)

        # Direction based on price movement
        if i > 0:
            direction = np.sign(prices[i] - prices[i - 1]) if prices[i] != prices[i - 1] else 0
        else:
            direction = 0

        trade_data.append(
            {
                "timestamp": date + pd.Timedelta(seconds=np.random.randint(0, 60)),
                "price": trade_price,
                "size": trade_size,
                "direction": direction,
            }
        )

trades_df = pd.DataFrame(trade_data).sort_values("timestamp")
trades_df = (
    trades_df.groupby("timestamp")
    .agg({"price": "mean", "size": "sum", "direction": "mean"})
    .reset_index()
)

# Add derived features
trades_df["returns"] = trades_df["price"].pct_change()
trades_df["log_returns"] = np.log(trades_df["price"] / trades_df["price"].shift(1))

print(f"Generated trade data: {len(trades_df)} aggregated trades")
trades_df.head()

In [ ]:
# Generate synthetic options data
options_data = []

for date, price in zip(dates[::60], prices[::60]):  # Hourly snapshots
    # Multiple maturities
    for days_to_expiry in [7, 14, 30, 60, 90]:
        # Multiple strikes
        for moneyness in np.arange(0.9, 1.11, 0.02):
            strike = price * moneyness

            # Implied volatility with skew
            base_iv = 0.20 + 0.10 * np.sin(len(options_data) * 0.01)  # Time-varying
            skew = 0.02 * (1.0 - moneyness)  # Negative skew
            iv = base_iv + skew

            # Term structure
            term_premium = 0.01 * np.sqrt(days_to_expiry / 30)
            iv += term_premium

            # Greeks (simplified)
            delta = moneyness if moneyness < 1 else 1 - (moneyness - 1)
            gamma = 0.05 / (abs(1 - moneyness) + 0.1)
            vega = 0.1 * np.sqrt(days_to_expiry / 365)

            options_data.append(
                {
                    "timestamp": date,
                    "strike": strike,
                    "days_to_expiry": days_to_expiry,
                    "moneyness": moneyness,
                    "iv": iv,
                    "delta": delta,
                    "gamma": gamma,
                    "vega": vega,
                    "underlying_price": price,
                }
            )

options_df = pd.DataFrame(options_data)

print(f"Generated options data: {len(options_df)} option snapshots")
print(f"Unique timestamps: {options_df['timestamp'].nunique()}")
print(f"Strikes per timestamp: {len(options_df) // options_df['timestamp'].nunique()}")
options_df.head(10)

### 1.2 Construct Trade Data

Generate trade-level data from OHLCV bars with price, volume, and direction.

In [ ]:
# Initialize microstructure feature engineer
micro_config = {
    "imbalance": {"levels": [1, 5, 10]},
    "spread": {"normalize": True},
    "vpin": {"window": 50},
    "volatility": {"frequencies": [5, 20, 60]},
}

micro_engineer = MicrostructureFeatureEngineer(config=micro_config)

# Compute order book imbalance
imbalance_features = micro_engineer.compute_order_book_imbalance(ob_pivot, levels=[1, 5, 10])

print("Order Book Imbalance Features:")
print(imbalance_features.describe())
imbalance_features.head()

In [ ]:
# Compute microprice
microprice_features = micro_engineer.compute_microprice(ob_pivot)

print("Microprice Features:")
print(microprice_features.describe())
microprice_features.head()

In [ ]:
# Compute spread dynamics
spread_features = micro_engineer.compute_spread_dynamics(ob_pivot)

print("Spread Dynamics Features:")
print(spread_features.describe())
spread_features.head()

In [ ]:
# Compute VPIN (trade flow toxicity)
# Merge trade data with order book
merged_data = pd.merge_asof(
    trades_df.sort_values("timestamp"),
    ob_pivot[["timestamp", "total_bid_size", "total_ask_size"]].sort_values("timestamp"),
    on="timestamp",
    direction="nearest",
)

vpin_features = micro_engineer.compute_vpin(merged_data, window=50)

print("VPIN (Flow Toxicity) Features:")
print(vpin_features.describe())
vpin_features.head()

In [ ]:
# Compute realized volatility at multiple frequencies
vol_features = micro_engineer.compute_realized_volatility(
    trades_df, frequencies=[5, 20, 60], price_col="price"
)

print("Realized Volatility Features:")
print(vol_features.describe())
vol_features.head()

In [ ]:
# Combine all microstructure features
micro_features = pd.concat(
    [imbalance_features, microprice_features, spread_features, vpin_features, vol_features], axis=1
)

# Remove duplicate timestamp column if any
micro_features = micro_features.loc[:, ~micro_features.columns.duplicated()]

print(f"\n📊 Combined Microstructure Features: {micro_features.shape}")
print(f"Features: {list(micro_features.columns)[:10]}...")
micro_features.head()

## 3. Compute Options-Implied Features

Using the `OptionsFeatureEngineer` class to extract:
- IV skew across strikes
- Term structure slope and curvature
- Vol-of-vol (volatility of implied volatility)
- Volatility smile curvature
- Greeks exposure

In [ ]:
# Initialize options feature engineer
options_config = {
    "skew": {"maturities": [30, 60]},
    "term_structure": {"atm_range": 0.05},
    "vol_of_vol": {"window": 20},
    "greeks": {"enabled": True},
}

options_engineer = OptionsFeatureEngineer(config=options_config)

# Compute IV skew
skew_features = options_engineer.compute_iv_skew(options_df, maturities=[30, 60])

print("IV Skew Features:")
print(skew_features.describe())
skew_features.head()

In [ ]:
# Compute term structure
term_features = options_engineer.compute_term_structure(options_df, atm_range=0.05)

print("Term Structure Features:")
print(term_features.describe())
term_features.head()

In [ ]:
# Compute vol-of-vol
vov_features = options_engineer.compute_vol_of_vol(options_df, maturities=[30, 60], window=20)

print("Vol-of-Vol Features:")
print(vov_features.describe())
vov_features.head()

In [ ]:
# Compute Greeks exposure
greeks_features = options_engineer.compute_greeks_exposure(options_df, maturity=30)

print("Greeks Exposure Features:")
print(greeks_features.describe())
greeks_features.head()

In [ ]:
# Combine all options features
options_features = pd.concat([skew_features, term_features, vov_features, greeks_features], axis=1)

# Remove duplicates
options_features = options_features.loc[:, ~options_features.columns.duplicated()]

print(f"\n📊 Combined Options Features: {options_features.shape}")
print(f"Features: {list(options_features.columns)}")
options_features.head()

## 4. Merge All Features

Combine microstructure and options features into a single dataset aligned by timestamp.

In [ ]:
# Merge microstructure and options features
# Options data is at lower frequency, so merge_asof with nearest
all_features = pd.merge_asof(
    micro_features.sort_values("timestamp"),
    options_features.sort_values("timestamp"),
    on="timestamp",
    direction="nearest",
    tolerance=pd.Timedelta("1H"),  # Max 1 hour gap
)

# Add price and returns from trades
all_features = pd.merge_asof(
    all_features.sort_values("timestamp"),
    trades_df[["timestamp", "price", "returns", "log_returns"]].sort_values("timestamp"),
    on="timestamp",
    direction="nearest",
)

# Set timestamp as index
all_features = all_features.set_index("timestamp")

# Drop rows with too many NaNs
all_features = all_features.dropna(thresh=len(all_features.columns) * 0.7)

print(f"\n✅ Final Feature Dataset: {all_features.shape}")
print(f"Time range: {all_features.index.min()} to {all_features.index.max()}")
print(f"Features: {len(all_features.columns)}")
print(f"\nMissing values per column:")
print(all_features.isnull().sum()[all_features.isnull().sum() > 0])

all_features.head()

## 5. Feature Distributions & Statistics

Analyze the statistical properties and distributions of engineered features.

In [ ]:
# Summary statistics
print("📊 Feature Summary Statistics:\n")
print(all_features.describe().T)

In [ ]:
# Plot distributions of key microstructure features
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle("Microstructure Feature Distributions", fontsize=16, y=1.02)

micro_cols = [
    "imbalance_level_1",
    "imbalance_level_5",
    "vpin",
    "relative_spread",
    "realized_vol_20",
    "microprice",
]

for idx, col in enumerate(micro_cols):
    ax = axes[idx // 3, idx % 3]
    if col in all_features.columns:
        all_features[col].hist(bins=50, ax=ax, edgecolor="black", alpha=0.7)
        ax.set_title(col)
        ax.set_xlabel("Value")
        ax.set_ylabel("Frequency")
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot distributions of key options features
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle("Options Feature Distributions", fontsize=16, y=1.02)

options_cols = ["iv_skew_30d", "term_slope", "vol_of_vol_30d", "delta_exposure"]

for idx, col in enumerate(options_cols):
    ax = axes[idx // 2, idx % 2]
    if col in all_features.columns:
        all_features[col].dropna().hist(bins=50, ax=ax, edgecolor="black", alpha=0.7, color="coral")
        ax.set_title(col)
        ax.set_xlabel("Value")
        ax.set_ylabel("Frequency")
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Feature Correlation Analysis

Identify multicollinearity and relationships between features.

In [ ]:
# Correlation matrix
correlation_matrix = all_features.corr()

# Plot full correlation heatmap
plt.figure(figsize=(16, 14))
sns.heatmap(
    correlation_matrix,
    cmap="RdBu_r",
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"label": "Correlation"},
    annot=False,
)
plt.title("Feature Correlation Heatmap", fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Find highly correlated features (potential redundancy)
high_corr_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append(
                {
                    "Feature 1": correlation_matrix.columns[i],
                    "Feature 2": correlation_matrix.columns[j],
                    "Correlation": correlation_matrix.iloc[i, j],
                }
            )

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values(
        "Correlation", key=abs, ascending=False
    )
    print("⚠️ Highly Correlated Feature Pairs (|r| > 0.8):\n")
    print(high_corr_df)
else:
    print("✅ No highly correlated feature pairs found (|r| > 0.8)")

In [ ]:
# Interactive correlation plot with plotly
fig = go.Figure(
    data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        colorscale="RdBu_r",
        zmid=0,
        zmin=-1,
        zmax=1,
        text=correlation_matrix.values,
        hovertemplate="%{x}<br>%{y}<br>Correlation: %{z:.3f}<extra></extra>",
        colorbar=dict(title="Correlation"),
    )
)

fig.update_layout(
    title="Interactive Feature Correlation Matrix",
    width=900,
    height=800,
    xaxis={"side": "bottom"},
    yaxis={"autorange": "reversed"},
)

fig.show()

### 1.3 Load Options Data

Fetch real options chains with implied volatilities and Greeks from OpenBB.

In [ ]:
# Plot time series of key features
fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=[
        "Price",
        "Order Book Imbalance",
        "VPIN (Flow Toxicity)",
        "IV Skew",
        "Realized Volatility",
        "Term Structure",
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1,
)

# Price
if "price" in all_features.columns:
    fig.add_trace(
        go.Scatter(
            x=all_features.index, y=all_features["price"], name="Price", line=dict(color="blue")
        ),
        row=1,
        col=1,
    )

# Imbalance
if "imbalance_level_1" in all_features.columns:
    fig.add_trace(
        go.Scatter(
            x=all_features.index,
            y=all_features["imbalance_level_1"],
            name="Imbalance L1",
            line=dict(color="green"),
        ),
        row=1,
        col=2,
    )

# VPIN
if "vpin" in all_features.columns:
    fig.add_trace(
        go.Scatter(
            x=all_features.index, y=all_features["vpin"], name="VPIN", line=dict(color="red")
        ),
        row=2,
        col=1,
    )

# IV Skew
if "iv_skew_30d" in all_features.columns:
    fig.add_trace(
        go.Scatter(
            x=all_features.index,
            y=all_features["iv_skew_30d"],
            name="IV Skew 30d",
            line=dict(color="purple"),
        ),
        row=2,
        col=2,
    )

# Realized Vol
if "realized_vol_20" in all_features.columns:
    fig.add_trace(
        go.Scatter(
            x=all_features.index,
            y=all_features["realized_vol_20"],
            name="Realized Vol 20",
            line=dict(color="orange"),
        ),
        row=3,
        col=1,
    )

# Term Structure
if "term_slope" in all_features.columns:
    fig.add_trace(
        go.Scatter(
            x=all_features.index,
            y=all_features["term_slope"],
            name="Term Slope",
            line=dict(color="teal"),
        ),
        row=3,
        col=2,
    )

fig.update_layout(title_text="Feature Time Series", height=900, showlegend=False)

fig.show()

In [ ]:
# Rolling statistics to check stability
window = 500  # ~8 hours of minute data

rolling_stats = pd.DataFrame(index=all_features.index)

# Select key features for stability analysis
key_features = ["imbalance_level_1", "vpin", "realized_vol_20", "iv_skew_30d"]

for feature in key_features:
    if feature in all_features.columns:
        rolling_stats[f"{feature}_mean"] = all_features[feature].rolling(window).mean()
        rolling_stats[f"{feature}_std"] = all_features[feature].rolling(window).std()

# Plot rolling mean and std
fig, axes = plt.subplots(len(key_features), 1, figsize=(14, 12))
fig.suptitle(f"Rolling Statistics (Window={window})", fontsize=16, y=0.995)

for idx, feature in enumerate(key_features):
    if feature in all_features.columns:
        ax = axes[idx]

        # Plot rolling mean
        ax.plot(
            rolling_stats.index,
            rolling_stats[f"{feature}_mean"],
            label="Mean",
            color="blue",
            linewidth=1.5,
        )

        # Plot rolling std as shaded area
        ax2 = ax.twinx()
        ax2.fill_between(
            rolling_stats.index,
            0,
            rolling_stats[f"{feature}_std"],
            alpha=0.3,
            color="red",
            label="Std Dev",
        )

        ax.set_title(feature, fontsize=12)
        ax.set_ylabel("Mean", color="blue")
        ax2.set_ylabel("Std Dev", color="red")
        ax.grid(alpha=0.3)

        if idx == len(key_features) - 1:
            ax.set_xlabel("Time")

plt.tight_layout()
plt.show()

## 8. Data Quality Validation

Check for constant features, extreme values, missing data, and other quality issues.

In [ ]:
# Check for constant or near-constant features
variance_check = all_features.var()
low_variance = variance_check[variance_check < 1e-10]

if len(low_variance) > 0:
    print("⚠️ Low Variance Features (potential constants):")
    print(low_variance)
else:
    print("✅ All features have sufficient variance")

print(f"\nVariance Statistics:")
print(variance_check.describe())

In [ ]:
# Load real options data using OpenBB
print(f"🎯 Loading options data for {SYMBOL}...")

try:
    # Load options chain
    options_df = loader.load_openbb_options(
        symbol=SYMBOL,
        provider="cboe",  # or "tradier", "intrinio" with API keys
    )

    print(f"✅ Loaded {len(options_df)} options contracts from OpenBB")
    print(f"   Columns: {list(options_df.columns)[:10]}...")

    # Process options data
    if "expiration" in options_df.columns:
        options_df["days_to_expiry"] = (
            pd.to_datetime(options_df["expiration"]) - pd.Timestamp.now()
        ).dt.days

    # Calculate moneyness
    current_price = (
        trades_df["price"].iloc[-1] if len(trades_df) > 0 else prices_df["close"].iloc[-1]
    )

    if "strike" in options_df.columns:
        options_df["moneyness"] = options_df["strike"] / current_price

    # Add timestamp
    if "timestamp" not in options_df.columns:
        options_df["timestamp"] = pd.Timestamp.now()

    # Rename IV column if needed
    iv_col = next(
        (col for col in options_df.columns if "iv" in col.lower() or "implied" in col.lower()), None
    )
    if iv_col and iv_col != "iv":
        options_df["iv"] = options_df[iv_col]

    print(
        f"   Expirations available: {options_df['expiration'].nunique() if 'expiration' in options_df.columns else 'N/A'}"
    )

except Exception as e:
    print(f"⚠️ Could not load real options data: {e}")
    print("   Generating synthetic options data as fallback...")

    # Fallback: Generate synthetic options data
    options_data = []
    current_price = trades_df["price"].iloc[-1] if len(trades_df) > 0 else 100

    # Sample timestamps (daily snapshots)
    sample_timestamps = pd.date_range(
        start=prices_df["timestamp"].min(), end=prices_df["timestamp"].max(), freq="1D"
    )

    for timestamp in sample_timestamps:
        # Get price at this timestamp
        price_row = (
            prices_df[prices_df["timestamp"] <= timestamp].iloc[-1] if len(prices_df) > 0 else None
        )
        price = (
            price_row["close"] if price_row is not None and "close" in price_row else current_price
        )

        # Multiple maturities
        for days_to_expiry in [7, 14, 30, 60, 90]:
            # Multiple strikes
            for moneyness in np.arange(0.9, 1.11, 0.02):
                strike = price * moneyness

                # Implied volatility with skew
                base_iv = 0.20  # 20% base IV
                skew = 0.02 * (1.0 - moneyness)  # Negative skew
                iv = base_iv + skew

                # Term structure
                term_premium = 0.01 * np.sqrt(days_to_expiry / 30)
                iv += term_premium

                # Greeks (simplified)
                delta = moneyness if moneyness < 1 else 1 - (moneyness - 1)
                gamma = 0.05 / (abs(1 - moneyness) + 0.1)
                vega = 0.1 * np.sqrt(days_to_expiry / 365)

                options_data.append(
                    {
                        "timestamp": timestamp,
                        "strike": strike,
                        "days_to_expiry": days_to_expiry,
                        "moneyness": moneyness,
                        "iv": iv,
                        "delta": delta,
                        "gamma": gamma,
                        "vega": vega,
                        "underlying_price": price,
                    }
                )

    options_df = pd.DataFrame(options_data)
    print(f"✅ Generated {len(options_df)} synthetic options contracts")

print(f"\nOptions data summary:")
print(f"   Unique timestamps: {options_df['timestamp'].nunique()}")
if "strike" in options_df.columns and "timestamp" in options_df.columns:
    print(f"   Strikes per timestamp: {len(options_df) // options_df['timestamp'].nunique()}")

options_df.head(10)

In [ ]:
# Generate trade-level data from OHLCV bars
print(f"📈 Constructing trade data from price bars...")

trade_data = []

for idx, row in prices_df.iterrows():
    timestamp = row["timestamp"]
    close = row["close"] if "close" in row else row["Close"]
    volume = row["volume"] if "volume" in row else row["Volume"]

    # Estimate number of trades per bar (varies by liquidity)
    n_trades = max(1, int(volume / 100))  # Rough approximation
    n_trades = min(n_trades, 50)  # Cap to avoid memory issues

    # Generate individual trades within the bar
    for i in range(n_trades):
        # Price variation within the bar
        trade_price = close * (1 + np.random.normal(0, 0.0001))
        trade_size = volume / n_trades * np.random.uniform(0.5, 1.5)

        # Add microsecond offset within the bar
        trade_time = timestamp + pd.Timedelta(seconds=(60 * i / n_trades))

        trade_data.append(
            {
                "timestamp": trade_time,
                "price": trade_price,
                "size": trade_size,
            }
        )

trades_df = pd.DataFrame(trade_data).sort_values("timestamp")

# Aggregate to reduce data size if needed
trades_df = (
    trades_df.groupby(pd.Grouper(key="timestamp", freq="1min"))
    .agg(
        {
            "price": "mean",
            "size": "sum",
        }
    )
    .reset_index()
)

# Compute trade direction using tick rule
trades_df["direction"] = np.sign(trades_df["price"].diff())
trades_df.loc[trades_df["direction"] == 0, "direction"] = np.nan
trades_df["direction"] = trades_df["direction"].ffill().fillna(0)

# Add returns
trades_df["returns"] = trades_df["price"].pct_change()
trades_df["log_returns"] = np.log(trades_df["price"] / trades_df["price"].shift(1))

print(f"✅ Constructed trade data: {len(trades_df)} aggregated trades")
print(f"   Average trade size: {trades_df['size'].mean():.2f}")
print(f"   Price range: ${trades_df['price'].min():.2f} - ${trades_df['price'].max():.2f}")

trades_df.head()

## 1. Load Real Market Data via OpenBB Terminal

Load historical price data, construct order book, and prepare trade-level data.

**Data Provider**: OpenBB Terminal (Yahoo Finance, Polygon, Alpha Vantage, etc.)  
**Symbol**: Configurable (default: SPY)  
**Frequency**: 1-minute bars for microstructure analysis

In [ ]:
## 0. Setup & Imports

Configure the environment and import required libraries.

# Feature Engineering for Volatility Trading with OpenBB

This notebook loads **real market data** using OpenBB Terminal and engineers features for RL trading models.

## Data Sources
- **Historical Prices**: OHLCV data from Yahoo Finance, Polygon, or Alpha Vantage
- **Order Book**: Approximated from OHLCV (or real Level 2 with Polygon API)
- **Options Data**: Real options chains with IVs and Greeks from CBOE/Tradier

## Features Engineered

**Microstructure Features:**
- Order book imbalance (levels 1, 5, 10)
- Microprice (volume-weighted mid)
- Spread dynamics (absolute, relative, normalized)
- VPIN (Volume-Synchronized Probability of Informed Trading)
- Realized volatility (5, 20, 60 period windows)

**Options-Implied Features:**
- IV skew across strikes (30d, 60d maturities)
- Term structure slope and curvature
- Vol-of-vol (volatility of implied volatility)
- Volatility smile curvature
- Greeks exposure (delta, gamma, vega, theta, rho)

## Output
- Processed feature dataset ready for RL model training
- Feature distributions and correlation analysis
- Temporal stability checks
- Data quality validation